In [1]:
from sedona.spark import *
from sedona.utils.adapter import Adapter
from sedona.core.enums import GridType, IndexType
from sedona.core.spatialOperator import JoinQuery
from sedona.sql import ST_IsValid, ST_MakeValid

from pyspark.sql.types import BooleanType
from pyspark.sql.functions import col, udf

In [2]:
config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.6.1,'
           'org.datasyslab:geotools-wrapper:1.6.1-28.2'). \
    getOrCreate()

sedona = SedonaContext.create(config)

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.sedona#sedona-spark-shaded-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3637918f-3dcb-4189-a9a9-65c26a658698;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.6.1 in central
	found org.datasyslab#geotools-wrapper;1.6.1-28.2 in central
:: resolution report :: resolve 115ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.6.1 from central in [default]
	org.datasyslab#geotools-wrapper;1.6.1-28.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      de

In [3]:
buildings_gdf = sedona.read.format("shapefile").option("recursiveFileLookup", "true").load("../ALKIS/*/GebauedeBauwerk.shp").dropDuplicates(["oid"])
nutzung_gdf = sedona.read.format("shapefile").option("recursiveFileLookup", "true").load("../ALKIS/*/NutzungFlurstueck.shp").dropDuplicates(["oid"])

In [4]:
buildings_gdf.count()

2473571

In [5]:
nutzung_gdf.count()

5243004

In [6]:
buildings_gdf.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- oid: string (nullable = true)
 |-- aktualit: string (nullable = true)
 |-- gebnutzbez: string (nullable = true)
 |-- funktion: string (nullable = true)
 |-- gfkzshh: string (nullable = true)
 |-- rellage: string (nullable = true)
 |-- name: string (nullable = true)
 |-- anzahlgs: long (nullable = true)
 |-- gmdschl: string (nullable = true)
 |-- lagebeztxt: string (nullable = true)



In [7]:
nutzung_gdf.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- oid: string (nullable = true)
 |-- aktualit: string (nullable = true)
 |-- nutzart: string (nullable = true)
 |-- bez: string (nullable = true)
 |-- name: string (nullable = true)
 |-- flaeche: decimal(12,2) (nullable = true)
 |-- flstkennz: string (nullable = true)



In [8]:
print(f"Buildings columns: {len(buildings_gdf.columns)}")
print(f"Nutzung columns: {len(nutzung_gdf.columns)}")

Buildings columns: 11
Nutzung columns: 8


In [9]:
nutzung_RDD = Adapter.toSpatialRdd(nutzung_gdf, "geometry")
buildings_RDD = Adapter.toSpatialRdd(buildings_gdf, "geometry")

In [17]:
nutzung_RDD.analyze()
buildings_RDD.analyze() 

nutzung_RDD.spatialPartitioning(GridType.KDBTREE)
buildings_RDD.spatialPartitioning(nutzung_RDD.getPartitioner())

In [18]:
build_on_spatial_partitioned_rdd = True ## Set to TRUE only if run join query
using_index = True

try:
    result_RDD = JoinQuery.SpatialJoinQueryFlat(buildings_RDD, nutzung_RDD, using_index, build_on_spatial_partitioned_rdd)
    print(f"Number of results: {result_RDD.count()}")
    print(result_RDD.take(5))  # Sample output
except Exception as e:
    print(f"An error occurred during the spatial join: {e}")

24/10/05 17:11:37 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


An error occurred during the spatial join: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 52.0 failed 1 times, most recent failure: Lost task 15.0 in stage 52.0 (TID 541) (28dd096dafc7 executor driver): org.locationtech.jts.geom.TopologyException: side location conflict [ (463724.13, 5796494.38, NaN) ]
	at org.locationtech.jts.geomgraph.EdgeEndStar.propagateSideLabels(EdgeEndStar.java:289)
	at org.locationtech.jts.geomgraph.EdgeEndStar.computeLabelling(EdgeEndStar.java:125)
	at org.locationtech.jts.operation.relate.RelateComputer.labelNodeEdges(RelateComputer.java:325)
	at org.locationtech.jts.operation.relate.RelateComputer.computeIM(RelateComputer.java:125)
	at org.locationtech.jts.operation.relate.RelateOp.getIntersectionMatrix(RelateOp.java:109)
	at org.locationtech.jts.operation.relate.RelateOp.relate(RelateOp.java:54)
	at org.locationtech.jts.geom.Geomet

24/10/05 17:11:47 ERROR Executor: Exception in task 15.0 in stage 52.0 (TID 541)
org.locationtech.jts.geom.TopologyException: side location conflict [ (463724.13, 5796494.38, NaN) ]
	at org.locationtech.jts.geomgraph.EdgeEndStar.propagateSideLabels(EdgeEndStar.java:289)
	at org.locationtech.jts.geomgraph.EdgeEndStar.computeLabelling(EdgeEndStar.java:125)
	at org.locationtech.jts.operation.relate.RelateComputer.labelNodeEdges(RelateComputer.java:325)
	at org.locationtech.jts.operation.relate.RelateComputer.computeIM(RelateComputer.java:125)
	at org.locationtech.jts.operation.relate.RelateOp.getIntersectionMatrix(RelateOp.java:109)
	at org.locationtech.jts.operation.relate.RelateOp.relate(RelateOp.java:54)
	at org.locationtech.jts.geom.Geometry.relate(Geometry.java:1035)
	at org.locationtech.jts.geom.Geometry.intersects(Geometry.java:767)
	at org.apache.sedona.core.spatialOperator.SpatialPredicateEvaluators$IntersectsEvaluator.eval(SpatialPredicateEvaluators.java:47)
	at org.apache.sedon

In [12]:
result_gdf = Adapter.toDf(result_RDD, sedona)
print(f"Result columns: {len(result_gdf.columns)}")

24/10/05 17:06:34 WARN TaskSetManager: Lost task 12.0 in stage 31.0 (TID 418) (28dd096dafc7 executor driver): TaskKilled (Stage cancelled)
Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 193, in manager
  File "/opt/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 874, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 193, in manager
  File "/opt/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 874, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 

Result columns: 19


In [13]:
result_gdf.printSchema()

root
 |-- _1: geometry (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: string (nullable = true)
 |-- _7: string (nullable = true)
 |-- _8: string (nullable = true)
 |-- _9: geometry (nullable = true)
 |-- _10: string (nullable = true)
 |-- _11: string (nullable = true)
 |-- _12: string (nullable = true)
 |-- _13: string (nullable = true)
 |-- _14: string (nullable = true)
 |-- _15: string (nullable = true)
 |-- _16: string (nullable = true)
 |-- _17: string (nullable = true)
 |-- _18: string (nullable = true)
 |-- _19: string (nullable = true)



In [14]:
# Add prefixes to distinguish between geometries, if needed
combined_columns = ["nutzung_" + col for col in nutzung_gdf.columns] + ["buildings_" + col for col in buildings_gdf.columns]

# Rename columns in result_gdf
for i, col_name in enumerate(combined_columns):
    result_gdf = result_gdf.withColumnRenamed(f"_{i + 1}", col_name)

result_gdf = result_gdf.drop("nutzung_geometry").withColumnRenamed("buildings_geometry", "geometry")
result_gdf.printSchema()

root
 |-- nutzung_oid: string (nullable = true)
 |-- nutzung_aktualit: string (nullable = true)
 |-- nutzung_nutzart: string (nullable = true)
 |-- nutzung_bez: string (nullable = true)
 |-- nutzung_name: string (nullable = true)
 |-- nutzung_flaeche: string (nullable = true)
 |-- nutzung_flstkennz: string (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- buildings_oid: string (nullable = true)
 |-- buildings_aktualit: string (nullable = true)
 |-- buildings_gebnutzbez: string (nullable = true)
 |-- buildings_funktion: string (nullable = true)
 |-- buildings_gfkzshh: string (nullable = true)
 |-- buildings_rellage: string (nullable = true)
 |-- buildings_name: string (nullable = true)
 |-- buildings_anzahlgs: string (nullable = true)
 |-- buildings_gmdschl: string (nullable = true)
 |-- buildings_lagebeztxt: string (nullable = true)



In [15]:
@udf(returnType=BooleanType())
def is_geometry_valid(wkt):
    try:
        from shapely import wkt as shapely_wkt
        from shape.errors import ShapelyError
        
        geom =  shapely_wkt.loads(wkt)
        return geom.is_valid
    except ShapelyError:
        return false
    except Exception:
        return False

In [16]:
valid_gdf = result_gdf.withColumn("is_geometry", is_geometry_valid(col("geometry")))
total_validity = valid_gdf.drop("geometry").count()

24/10/05 17:06:35 ERROR Executor: Exception in task 14.0 in stage 41.0 (TID 438)
org.locationtech.jts.geom.TopologyException: side location conflict [ (274565.7199458984, 5882096.033370846, NaN) ]
	at org.locationtech.jts.geomgraph.EdgeEndStar.propagateSideLabels(EdgeEndStar.java:289)
	at org.locationtech.jts.geomgraph.EdgeEndStar.computeLabelling(EdgeEndStar.java:125)
	at org.locationtech.jts.operation.relate.RelateComputer.labelNodeEdges(RelateComputer.java:325)
	at org.locationtech.jts.operation.relate.RelateComputer.computeIM(RelateComputer.java:125)
	at org.locationtech.jts.operation.relate.RelateOp.getIntersectionMatrix(RelateOp.java:109)
	at org.locationtech.jts.operation.relate.RelateOp.relate(RelateOp.java:54)
	at org.locationtech.jts.geom.Geometry.relate(Geometry.java:1035)
	at org.locationtech.jts.geom.Geometry.intersects(Geometry.java:767)
	at org.apache.sedona.core.spatialOperator.SpatialPredicateEvaluators$IntersectsEvaluator.eval(SpatialPredicateEvaluators.java:47)
	at o

Py4JJavaError: An error occurred while calling o146.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 14 in stage 41.0 failed 1 times, most recent failure: Lost task 14.0 in stage 41.0 (TID 438) (28dd096dafc7 executor driver): org.locationtech.jts.geom.TopologyException: side location conflict [ (274565.7199458984, 5882096.033370846, NaN) ]
	at org.locationtech.jts.geomgraph.EdgeEndStar.propagateSideLabels(EdgeEndStar.java:289)
	at org.locationtech.jts.geomgraph.EdgeEndStar.computeLabelling(EdgeEndStar.java:125)
	at org.locationtech.jts.operation.relate.RelateComputer.labelNodeEdges(RelateComputer.java:325)
	at org.locationtech.jts.operation.relate.RelateComputer.computeIM(RelateComputer.java:125)
	at org.locationtech.jts.operation.relate.RelateOp.getIntersectionMatrix(RelateOp.java:109)
	at org.locationtech.jts.operation.relate.RelateOp.relate(RelateOp.java:54)
	at org.locationtech.jts.geom.Geometry.relate(Geometry.java:1035)
	at org.locationtech.jts.geo

In [ ]:
(valid_gdf.write
 .format("geoparquet")
 .mode("overwrite")
 .option("geoparquet.version", "1.0.0")
 .save("sedona_data.geoparquet")
)